# ขั้นตอนทั้งหมด

## video to images

In [ ]:
!unzip /content/drive/MyDrive/Cars/Cars.zip

Archive:  /content/drive/MyDrive/Cars/Cars.zip
  inflating: Cars/MB4 IN 15.mp4      
  inflating: Cars/MB4 IN 13.mp4      
  inflating: Cars/MB4 IN 14.mp4      


In [ ]:
import cv2
import os

def extract_frames(video_path, output_dir, fps_interval=1):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * fps_interval)

    count = 0
    saved = 0
    basename = os.path.splitext(os.path.basename(video_path))[0]

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if count % frame_interval == 0:
            filename = f"{basename}_frame_{saved:05d}.jpg"
            filepath = os.path.join(output_dir, filename)
            cv2.imwrite(filepath, frame)
            saved += 1

        count += 1

    cap.release()
    print(f"[{basename}] ดึงเฟรมแล้วทั้งหมด {saved} ภาพ")

video_files = ["/content/Cars/MB4 IN 13.mp4", "/content/Cars/MB4 IN 14.mp4", "/content/Cars/MB4 IN 15.mp4"]

for v in video_files:
    extract_frames(v, output_dir="frames", fps_interval=1)


[MB4 IN 13] ดึงเฟรมแล้วทั้งหมด 3600 ภาพ
[MB4 IN 14] ดึงเฟรมแล้วทั้งหมด 3600 ภาพ
[MB4 IN 15] ดึงเฟรมแล้วทั้งหมด 2262 ภาพ


## sample500

In [ ]:
import random
import shutil

all_images_dir = "/content/frames/"
sample_output_dir = "sample_500/"
num_samples = 500

output_dir = "remaining_images/"
os.makedirs(output_dir, exist_ok=True)

# รายชื่อไฟล์ (ใช้เฉพาะชื่อ ไม่รวม path)
sample_images = set(os.listdir(sample_dir))
all_images = set(os.listdir(all_dir))

# หาภาพที่เหลือ
remaining_images = all_images - sample_images

# คัดลอกภาพที่เหลือไปยัง output_dir
for img in remaining_images:
    shutil.copy(os.path.join(all_dir, img), os.path.join(output_dir, img))

print(f"คัดลอกภาพที่เหลือเสร็จแล้ว: {len(remaining_images)} ภาพ")

สุ่มเสร็จแล้ว: 500 รูป -> sample_500/


In [ ]:
!zip -r /content/sample_500.zip /content/sample_500

  adding: content/sample_500/ (stored 0%)
  adding: content/sample_500/MB4 IN 15_frame_02206.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 13_frame_00485.jpg (deflated 0%)
  adding: content/sample_500/MB4 IN 14_frame_01728.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 15_frame_01257.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 13_frame_03209.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 14_frame_01800.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 14_frame_03508.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 14_frame_00417.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 15_frame_00391.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 14_frame_00758.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 14_frame_00045.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 14_frame_02824.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 13_frame_01503.jpg (deflated 1%)
  adding: content/sample_500/MB4 IN 15_frame_01575.jpg (defla

## sample 1500

In [ ]:
all_images_dir = "/content/frames/"
sample_dir = "/content/content/sample_500"
output_dir = "remaining_images/"
os.makedirs(output_dir, exist_ok=True)

# รายชื่อไฟล์ (ใช้เฉพาะชื่อ ไม่รวม path)
sample_images = set(os.listdir(sample_dir))
all_images = set(os.listdir(all_images_dir))

# หาภาพที่เหลือ
remaining_images = all_images - sample_images

# คัดลอกภาพที่เหลือไปยัง output_dir
for img in remaining_images:
    shutil.copy(os.path.join(all_images_dir, img), os.path.join(output_dir, img))

print(f"คัดลอกภาพที่เหลือเสร็จแล้ว: {len(remaining_images)} ภาพ")

คัดลอกภาพที่เหลือเสร็จแล้ว: 8962 ภาพ


In [ ]:
# ที่เก็บภาพทั้งหมด (ยังไม่มี label)
all_image_dir = "content/remaining_images"
subset_dir = "/content/sample1500/images"
os.makedirs(subset_dir, exist_ok=True)

# สุ่ม 1500
all_images = [f for f in os.listdir(all_image_dir) if f.endswith(".jpg")]
sample_images = random.sample(all_images, 1500)

# คัดลอกไปโฟลเดอร์สำหรับ labeling
for img_name in sample_images:
    shutil.copy(os.path.join(all_image_dir, img_name), os.path.join(subset_dir, img_name))

print("เตรียมภาพสำหรับ labeling แล้วที่ /sample1500/images (1500 ภาพ)")

## train model for auto Labeling

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

model.train(data="/content/sample500/data.yaml", epochs=20, imgsz=640)

## Auto Labeling

In [ ]:
import os
import shutil
from glob import glob
from ultralytics import YOLO

# โหลดโมเดล
model = YOLO("/content/models/model_auto_label.pt")

# โฟลเดอร์ input/output
input_dir = "/content/sample1500/images"
label_output_dir = "/content/sample1500/labels"
os.makedirs(label_output_dir, exist_ok=True)

# Predict แบบ batch
image_paths = sorted(glob(os.path.join(input_dir, "*.jpg")))
batch_size = 100

for i in range(0, len(image_paths), batch_size):
    batch = image_paths[i:i + batch_size]
    print(f"Predicting images {i} to {i + len(batch) - 1}")

    model.predict(
        source=batch,
        save=False,
        save_txt=True,
        project="/content/sample1500",
        name="labels",
        exist_ok=True,
        imgsz=640,
        conf=0.4
    )


shutil.make_archive("roboflow_ready", 'zip', "/content/sample1500")

print("เสร็จสิ้น! ได้ไฟล์ roboflow_ready.zip สำหรับอัปโหลดเข้า Roboflow")

## train model จริง

In [ ]:
model = YOLO("yolo11s.pt")

model.train(data="/content/roboflow_ready/data.yaml", epochs=20, imgsz=640)

## process video

In [ ]:
import threading, queue
import cv2
from ultralytics import YOLO
from PIL import ImageFont, ImageDraw, Image
import numpy as np

# โหลดโมเดล final_model
model = YOLO("/content/drive/MyDrive/Cars/best_02.pt")
# model.half()

# โหลด font
font = ImageFont.truetype("/content/THSarabun.ttf", 28)
class_names = ['รถยนต์', 'มอเตอร์ไซค์', 'รถกระบะ', 'รถกระบะ', 'รถบรรทุก', 'รถตู้']
class_colors = {
    'รถยนต์': (255, 255, 0),
    'มอเตอร์ไซค์': (0, 0, 255),
    'รถกระบะ': (0, 255, 0),
    'รถบรรทุก': (0, 255, 255),
    'รถตู้': (255, 0, 0),
}

# กำหนด ROI (ปรับค่าได้ตามต้องการ)
roi = (100, 460, 1000, 720)  # (x_min, y_min, x_max, y_max)

# เตรียม Queue
frame_queue = queue.Queue(maxsize=5)
counted_ids = set()
class_counts = {}

# เปิดวิดีโอ
cap = cv2.VideoCapture("/content/Cars/MB4 IN 13.mp4")

# เตรียมวิดีโอ output
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/content/drive/MyDrive/Cars/MB4 IN 13_output14.mp4", fourcc, fps, (width, height))

# Thread สำหรับอ่านวิดีโอ
def read_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            frame_queue.put(None)  # สัญญาณหยุด
            break
        frame_queue.put(frame)

reader_thread = threading.Thread(target=read_frames, daemon=True)
reader_thread.start()

while True:
    frame = frame_queue.get()
    if frame is None:
        break

    x_min, y_min, x_max, y_max = roi
    roi_frame = frame[y_min:y_max, x_min:x_max]

    results = model.track(roi_frame, persist=True, conf=0.4, imgsz=576)
    boxes = results[0].boxes

    pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)

    # วาดกรอบ ROI บนภาพเต็ม
    draw.rectangle([x_min, y_min, x_max, y_max], outline=(255, 0, 0), width=3)
    draw.text((x_min, y_min - 30), "ROI", font=font, fill=(255, 0, 0))

    if boxes.id is not None:
        ids = boxes.id.int().tolist()
        classes = boxes.cls.int().tolist()
        xyxy = boxes.xyxy.cpu().numpy()
        confidences = boxes.conf.tolist()

        for box, class_id, track_id, conf in zip(xyxy, classes, ids, confidences):
            x1, y1, x2, y2 = map(int, box)

            x1 += x_min
            x2 += x_min
            y1 += y_min
            y2 += y_min

            class_name = class_names[class_id]
            color = class_colors.get(class_name, (0, 255, 0))

            if track_id not in counted_ids:
                counted_ids.add(track_id)
                class_counts[class_name] = class_counts.get(class_name, 0) + 1

            draw.rectangle([x1, y1, x2, y2], outline=color[::-1], width=2)
            label = f"{class_name} {conf*100:.1f}%"#ID:{track_id}
            draw.text((x1 + 5, y1 - 30), label, font=font, fill=color[::-1])

    # แสดง counter
    y = 10
    for cls, count in class_counts.items():
        draw.text((10, y), f"{cls}: {count}", font=font, fill=(255, 255, 0))
        y += 35

    annotated = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    # cv2.imshow("Fast YOLO with ROI crop", annotated)

    out.write(annotated)

cap.release()
out.release()

print("เสร็จแล้ว!")